In [ ]:
import pandas as pd
import json
import requests

## Filtering eCheckbook


In [ ]:
# Load mbe and BAVN names.
final_mbe = pd.read_csv('equity-metrics\mbe\data\final_mbe.csv')

# Load manually matched names.
manually_matched = pd.read_csv('equity-metrics\mbe\data\manually_matched.csv')

In [ ]:
# Load all eCheckbook data for the specified fiscal year.
echeckbook = pd.DataFrame()

for i in np.arange(15):
    url = f'https://controllerdata.lacity.org/resource/pggv-e4fn.json?fiscal_year=2020&$limit=50000&$offset={str(i*50000)}'

    echeckbook_req = requests.get(url)
    echeckbook_json = json.loads(echeckbook_req.text)
    echeckbook_df = pd.DataFrame(echeckbook_json)

    echeckbook = echeckbook.append(echeckbook_df, ignore_index=True)

echeckbook

In [ ]:
# Put all mbe names into a list used to filter the eCheckbook data.
mbe_names = final_mbe[['MBE_COMPANY_NAME', 'FIRST_BAVN_NAME', 'SECOND_BAVN_NAME', 'BAVN_DBA']].to_numpy().reshape(-1)
mbe_names = mbe_names[~pd.isna(mbe_names)]

# Add the manually_matched names to mbe_names and filter for unique names.
manually_matched = manually_matched.to_numpy().reshape(-1)
mbe_names = set(mbe_names + manually_matched[~pd.isna(manually_matched)])

In [ ]:
# Filter eCheckbook data and save it.
filtered_echeckbook = echeckbook[echeckbook['vendor_name'].isin(mbe_names)]
filtered_echeckbook.to_csv('equity-metrics\mbe\outputs\filtered_echeckbook_2020.csv')